In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np


In [2]:
np.arange(0, 1.1, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [32]:
fn_results = "./data/results_neu.parquet"

In [33]:
def get_total_results(fn_results: str) -> pd.DataFrame:
    """Get results aggregate over all periods

    Args:
        fn_results: name of file with hourly results
    """
    # settings for aggregation
    cols = [
        "nuclear",
        "renewable",
        "netStorage",
        "demand",
        "energyNotServed",
        "curtailNuclear",
        "curtailRenewable",
    ]
    idx = [
        "share_storage",
        "share_generation",
        "share_renewable",
        "curtailRenewableFirst",
    ]
    # get results, add indicator which technology is dispatched first, and aggregate
    df_annual = (
        pd.read_parquet(fn_results)
        .assign(
            curtailRenewableFirst=lambda df: df["costCurtailRenewable"]
            <= df["costCurtailNuclear"],
        )
        .groupby(idx, as_index=False)[cols]
        .sum()
    )
    to_round =         [
            "share_storage",
            "share_generation",
            "share_renewable",
        ]


    # curtailment in percent of total generation
    df_annual = df_annual.assign(
        curtailNuclearPercent=lambda df: df["curtailNuclear"]
        / (df["nuclear"] + df["curtailNuclear"])
        * 100,
        curtailRenewablePercent=lambda df: df["curtailRenewable"]
        / (df["renewable"] + df["curtailRenewable"])
        * 100,
    )
    return df_annual.fillna(0)


df_annual = get_total_results(fn_results)
df_annual

,share_storage,share_generation,share_renewable,curtailRenewableFirst,nuclear,renewable,netStorage,demand,energyNotServed,curtailNuclear,curtailRenewable,curtailNuclearPercent,curtailRenewablePercent
0,0.00000,1.0,0.0,False,92.339875,0.000000,0.000000e+00,100.0,7.660125,7.660125,0.000000,7.660125,0.000000
1,0.00000,1.0,0.0,True,92.339875,0.000000,0.000000e+00,100.0,7.660125,7.660125,0.000000,7.660125,0.000000
2,0.00000,1.0,0.1,False,82.177926,10.000000,0.000000e+00,100.0,7.822074,7.822074,0.000000,8.691193,0.000000
3,0.00000,1.0,0.1,True,86.578944,5.598983,0.000000e+00,100.0,7.822074,3.421056,4.401017,3.801174,44.010172
4,0.00000,1.0,0.2,False,71.026265,20.000000,0.000000e+00,100.0,8.973735,8.973735,0.000000,11.217169,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,0.00005,1.2,0.8,True,24.000000,50.645119,-5.204170e-18,100.0,4.224226,0.000000,45.354881,0.000000,47.244668
656,0.00005,1.2,0.9,False,4.890343,70.445961,-2.168404e-19,100.0,6.417845,7.109657,37.554039,59.247143,34.772258
657,0.00005,1.2,0.9,True,12.000000,61.210707,-2.168404e-19,100.0,6.417845,0.000000,46.789293,0.000000,43.323420
658,0.00005,1.2,1.0,False,0.000000,73.056467,0.000000e+00,100.0,8.974910,0.000000,46.943533,0.000000,39.119610


In [37]:
# inputs
curtail_res_first = True
cost_res = 2
cost_nuc = 1
cost_sto = 10
cost_ens = 100
share_generation = 1.2
variable = "cost"


def get_plot_variable(
    df_annual: pd.DataFrame,
    cost_res: float,
    cost_nuc: float,
    cost_sto: float,
    cost_ens: float,
    share_generation: float,
    curtail_res_first=True,
) -> pd.DataFrame:
    """Get data for plotting depending and calculate cost
    Args:
        df_annual: results aggregated over the whole time horizon
        cost_res: cost to install renewable generation [Euro/MWh]
        cost_nuc: cost to install nuclear generation [Euro/MWh]
        cost_res: cost to install storage facility [Euro/MWh]
        cost_res: cost to install energy not served [Euro/MWh]
        share_generation: total generation as multiple of demand
        curtail_res_first: indicator whether renewable are curtailed first
    """
    df_annual
    df = (
        df_annual.query(f"share_generation == {share_generation}")
        .query(f"curtailRenewableFirst == {curtail_res_first}")
        .assign(
            cost=lambda df: (
                (df["nuclear"] + df["curtailNuclear"]) * cost_nuc
                + (df["renewable"] + df["curtailRenewable"]) * cost_res
                + df["energyNotServed"] * cost_ens
                + df["demand"] * df["share_storage"] * cost_sto
            )
        )
        .drop(["curtailRenewableFirst", "share_generation"], axis=1)
        .set_index(["share_storage", "share_renewable"])
        .reset_index()
    )
    print(df.groupby("share_storage").share_renewable.count())
    return df


df_cost = get_plot_variable(
    df_annual,
    cost_res=cost_res,
    cost_nuc=cost_nuc,
    cost_sto=cost_sto,
    cost_ens=cost_ens,
    share_generation=1,
    curtail_res_first=True,
)
df_cost  # .pivot_table("energyNotServed", "share_storage", "share_renewable")

share_storage
0.00000    11
0.00001    11
0.00002    11
0.00003    11
0.00004    11
0.00005    11
Name: share_renewable, dtype: int64


,share_storage,share_renewable,nuclear,renewable,netStorage,demand,energyNotServed,curtailNuclear,curtailRenewable,curtailNuclearPercent,curtailRenewablePercent,cost
0,0.00000,0.0,92.339875,0.000000,0.0,100.0,7.660125,7.660125,0.000000,7.660125,0.000000,866.012487
1,0.00000,0.1,86.578944,5.598983,0.0,100.0,7.822074,3.421056,4.401017,3.801174,44.010172,892.207351
2,0.00000,0.2,79.060367,11.965898,0.0,100.0,8.973735,0.939633,8.034102,1.174541,40.170509,1017.373493
3,0.00000,0.3,69.879343,19.129034,0.0,100.0,10.991623,0.120657,10.870966,0.172367,36.236555,1229.162307
4,0.00000,0.4,60.000000,26.487361,0.0,100.0,13.512639,0.000000,13.512639,0.000000,33.781598,1491.263911
...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.00005,0.6,40.000000,32.555755,0.0,100.0,2.714790,0.000000,27.444245,0.000000,45.740409,431.529033
62,0.00005,0.7,30.000000,40.611660,0.0,100.0,4.280355,0.000000,29.388340,0.000000,41.983343,598.085467
63,0.00005,0.8,20.000000,49.129149,0.0,100.0,6.184726,0.000000,30.870851,0.000000,38.588563,798.522553
64,0.00005,0.9,10.000000,57.844774,0.0,100.0,8.374912,0.000000,32.155226,0.000000,35.728029,1027.541210


In [67]:
df_plot = df_cost.copy()
df = df_plot.pivot_table(
    variable,
    "share_storage",
    "share_renewable",
)
print(df.values.shape, len(df.index), len(df.columns))
fig = px.imshow(
    df.values,
    x=df.columns,
    y=df.index,
    labels=dict(
        x="Renewable share [%]",
        y="Storage size [% of total demand]",
        color=f"{variable}",
    ),
    aspect="auto",
)
fig.show()

(3, 11) 3 11


In [63]:
df.values

array([[ 375.33333333,  200.        ,  280.        ],
       [ 390.73333333,  210.        ,  290.        ],
       [ 439.13333333,  220.        ,  300.        ],
       [ 516.4       ,  230.        ,  310.        ],
       [ 610.43333333,  240.        ,  320.        ],
       [ 713.93333333,  250.        ,  330.        ],
       [ 822.        ,  260.        ,  340.        ],
       [ 932.5       ,  270.        ,  350.        ],
       [1044.56666667,  280.        ,  360.        ],
       [1157.76666667,  290.        ,  370.        ],
       [1271.66666667,  300.        ,  380.        ]])

In [71]:
def plot_heatmap(df_plot: pd.DataFrame, variable: str = "cost") -> go.Figure:
    """Plot heat map for a given variable

    Args:
        df_plot: basic data for plotting already filtered for generation share
        variable: variable to plot
    """
    df = df_plot.pivot_table(
        variable,
        "share_storage",
        "share_renewable",
    )
    fig = px.imshow(
        df.values,
        x=list(df.columns),
        y=list(df.index),
        labels=dict(
            y="Renewable share [%]",
            x="Storage size [% of total demand]",
            color=f"{variable}",
        ),
        aspect="auto",
    )
    return fig


fig = plot_heatmap(df_cost)
fig.show()

In [57]:
df.index

Index([0.0, 0.1, 0.2, 0.3, 0.4000000000000001, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], dtype='float64', name='share_renewable')

In [31]:
fig = go.Figure(
    data=[go.Surface(z=df.values, x=df.index, y=df.columns)],
)
fig.update_layout(
    scene=dict(
        xaxis_title="x: Renewable share [%]",
        yaxis_title="y: Storage size [% of total demand]",
        zaxis_title=f"z: {variable}",
    ),
    autosize=True,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

In [32]:
fig = px.imshow(
    df.values,
    x=df.index,
    y=df.columns,
    labels=dict(
        x="Renewable share [%]", 
        y="Storage size [% of total demand]", 
        color=f"{variable}"
        ),
    aspect="auto"
)
fig.show()

In [33]:
df

share_storage,0.00,0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20
share_renewable,,,,,,,,,,,
0.0,449.577104,130.0,150.0,170.0,190.0,210.0,230.0,250.0,270.0,290.0,310.0
0.1,488.894047,141.0,161.0,181.0,201.0,221.0,241.0,261.0,281.0,301.0,321.0
0.2,639.246736,152.0,172.0,192.0,212.0,232.0,252.0,272.0,292.0,312.0,332.0
0.3,854.617332,163.0,183.0,203.0,223.0,243.0,263.0,283.0,303.0,323.0,343.0
0.4,1119.491624,174.0,194.0,214.0,234.0,254.0,274.0,294.0,314.0,334.0,354.0
0.5,1421.781112,185.0,205.0,225.0,245.0,265.0,285.0,305.0,325.0,345.0,365.0
0.6,1748.058829,196.0,216.0,236.0,256.0,276.0,296.0,316.0,336.0,356.0,376.0
0.7,2085.385219,207.0,227.0,247.0,267.0,287.0,307.0,327.0,347.0,367.0,387.0
0.8,2429.207565,218.0,238.0,258.0,278.0,298.0,318.0,338.0,358.0,378.0,398.0
